## **분류 연습 문제**
___
출처 : 핸즈온 머신러닝 Ch03 분류 연습문제 1, 2번

In [6]:
# import data
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version = 1, as_frame = False)

In [7]:
X, y = mnist["data"], mnist["target"]

In [8]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

### **1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해보세요.**
___

1. `KNeghtborsClassifier`를 사용하는 것을 추천합니다.
2. `weights`와 `n_neighbors` 하이퍼 파라미터로 그리드 탐색을 시도하여, 좋은 하이퍼 파라미터 값을 찾아보세요.

In [9]:
# import package

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [10]:
# Try GridSearch to optimize hyperparameter

# MNIST 데이터 불러오기
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist["data"], mnist["target"]

# 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

# 특성 스케일링 (KNN 성능 향상)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# KNN 하이퍼파라미터 최적화
param_grid = {
    'n_neighbors': [3, 5, 7, 9],  # 이웃 개수 조정
    'weights': ['uniform', 'distance']  # 가중치 방식 선택
}

knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [3, 5, 7, 9],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [11]:
# best hyperparameter

best_params = grid_search.best_params_
print("최적 하이퍼파라미터:", best_params)

최적 하이퍼파라미터: {'n_neighbors': 5, 'weights': 'distance'}


In [12]:
# best score

best_score = grid_search.best_score_
print("최고 교차 검증 정확도: {:.2f}%".format(best_score * 100))

최고 교차 검증 정확도: 94.21%


In [13]:
# model test

# 최적 하이퍼파라미터를 사용하여 모델 생성
best_knn = grid_search.best_estimator_

# 테스트 세트에서 예측 수행
y_pred = best_knn.predict(X_test_scaled)

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred)
print("테스트 세트 정확도: {:.2f}%".format(accuracy * 100))

테스트 세트 정확도: 94.50%


### **2. 다음 단계를 따라 인위적으로 훈련 세트를 늘리는 데이터 증식 또는 훈련 세트 확장 기법을 연습해봅시다.**
___

#### **STEP 1. MNIST 이미지를 (왼, 오른, 위, 아래) 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어 보세요.**

In [14]:
import scipy.ndimage

def shift_image(image, direction):
    """이미지를 원하는 방향으로 한 픽셀 이동시키는 함수"""
    if direction == "left":
        return scipy.ndimage.shift(image.reshape(28, 28), (-1, 0), cval=0).reshape(-1)
    elif direction == "right":
        return scipy.ndimage.shift(image.reshape(28, 28), (1, 0), cval=0).reshape(-1)
    elif direction == "up":
        return scipy.ndimage.shift(image.reshape(28, 28), (0, -1), cval=0).reshape(-1)
    elif direction == "down":
        return scipy.ndimage.shift(image.reshape(28, 28), (0, 1), cval=0).reshape(-1)
    else:
        return image


####  **STEP 2. 앞에서 만든 함수를 이용하여, 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본(방향마다 한 개씩)을 만들어 훈련 세트에 추가하세요**

In [15]:
X_train_augmented = []
y_train_augmented = []

for image, label in zip(X_train, y_train):
    X_train_augmented.append(image)
    y_train_augmented.append(label)

    # 네 방향으로 이동한 이미지 추가
    for direction in ["left", "right", "up", "down"]:
        X_train_augmented.append(shift_image(image, direction))
        y_train_augmented.append(label)

# 리스트를 NumPy 배열로 변환
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

print("데이터 증강 완료! 새로운 훈련 샘플 수:", len(X_train_augmented))


데이터 증강 완료! 새로운 훈련 샘플 수: 300000


####  **STEP 3. 위에서 확장한 데이터셋을 이용하여, 1번 문제에서 찾은 최적 모델을 훈련시키고, 테스트 세트에서 정확도를 측정해보세요**

In [16]:
# 증강된 데이터 스케일링
X_train_augmented_scaled = scaler.fit_transform(X_train_augmented)

# 최적의 모델 재훈련
best_knn.fit(X_train_augmented_scaled, y_train_augmented)

# 증강된 데이터로 학습 후 테스트 데이터에서 평가
y_pred_augmented = best_knn.predict(X_test_scaled)

# 새로운 정확도 평가
accuracy_augmented = accuracy_score(y_test, y_pred_augmented)
print("증강된 데이터로 학습 후 테스트 세트 정확도: {:.2f}%".format(accuracy_augmented * 100))


증강된 데이터로 학습 후 테스트 세트 정확도: 95.76%
